# Assignment 3: Hidden Markov Models for POS Tagging and Text Generation
## CNG463 - Introduction to Natural Language Processing
### METU NCC Computer Engineering | Fall 2025-26

**Student Name:**  
**Student ID:**  
**Due Date:** 14 December 2025 (Sunday) before midnight

---

## Overview

This assignment focuses on:
1. Building **supervised**, **unsupervised**, and **semi-supervised** HMM models for Part-of-Speech (POS) tagging
2. Implementing **5-fold cross-validation** to evaluate model performance
3. Comparing the three training approaches using per-tag and overall accuracy
4. Using HMMs for **creative text generation** with temperature-based sampling

**Note:** You will use the Brown corpus with universal POS tagset. Start with 5000 sentences for debugging, then run on the full corpus (or as much as Colab can handle).

**Grading:**
- Helper Functions: **10 pts**
  - `remove_tags()`: 2 pts
  - `evaluate_tagger()`: 5 pts
  - Results display: 3 pts
- Semi-supervised HMM: **20 pts**
- 5-fold cross-validation: **25 pts**
  - Supervised HMM: 10 pts
  - Unsupervised HMM (Baum-Welch): 10 pts
  - Semi-supervised HMM
  - Evaluate and Accumulate Results: 5 pts
- Report on Results: **5 pts**
- Text Generation: **24 pts**
  - `sample_state()`: 7 pts
  - `sample_word()`: 7 pts
  - `generate_text_from_word()`: 10 pts
- Written Questions (4 × 4 pts): **16 pts**
- **Total: 100 pts**

---

## Pre-Submission Checklist

- [ ] Name and student ID at top
- [ ] No cells are added or removed
- [ ] All TODO sections completed
- [ ] All questions answered
- [ ] Code runs without errors
- [ ] Results tables included
- [ ] Run All before saving

## Setup and Imports

In [1]:
# Standard libraries
import numpy as np
import random
from collections import defaultdict

# NLTK for corpus and HMM
import nltk
from nltk.tag import hmm

# Scikit-learn for cross-validation
from sklearn.model_selection import KFold

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)

---

# Task 1: HMM-based POS Tagging (72 points)

In this part, you will implement three different approaches to training HMM models for POS tagging:
1. **Supervised learning**: Uses fully labelled data
2. **Unsupervised learning**: Uses unlabelled data with Baum-Welch algorithm
3. **Semi-supervised learning**: Combines a small amount of labelled data with larger unlabelled data

## 1.1: Load the Brown Corpus

Load the Brown corpus with universal POS tagset. Start with 5000 sentences for testing, then increase to the maximum your environment can handle.

In [2]:
# Download required NLTK data
from nltk.corpus import brown
nltk.download('brown')
nltk.download('universal_tagset')

NUM_SENTENCES = 5000  # Start with 5000 sentences and increase later

all_data = list(brown.tagged_sents(tagset="universal"))[:5000]

print(f"Total sentences loaded: {len(all_data)}")
print(f"\nExample sentence:")
print(all_data[0])

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


Total sentences loaded: 5000

Example sentence:
[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')]


## 1.2: Remove Tags (2 points)

Implement the `remove_tags()` function that converts tagged sentences to untagged format required by NLTK's unsupervised training.

**Input:** List of tagged sentences `[[(word, tag), ...], ...]`  
**Output:** List of untagged sentences `[[(word, None), ...], ...]`

In [3]:
def remove_tags(tagged_sents):
    """
    Remove tags from tagged sentences to create untagged data.

    Args:
        tagged_sents: List of tagged sentences [[(word, tag), ...], ...]

    Returns:
        List of untagged sentences [[(word, None), ...], ...] (required format for NLTK)
    """
    # TODO: Implement this function

    untagged_data = []

    # 2. Loop through every sentence in the input data
    for sentence in tagged_sents:

        # Create a temporary list for the current sentence
        new_sentence = []

        # 3. Loop through every word pair in the current sentence
        for word, tag in sentence:
            # Keep the word, but change the tag to None
            new_pair = (word, None)
            new_sentence.append(new_pair)

        # Add the processed sentence to our main list
        untagged_data.append(new_sentence)

    return untagged_data

## 1.3: Evaluate Tagger (5 points)

Implement the `evaluate_tagger()` function that evaluates a trained tagger on test data and returns per-tag accuracy statistics.

**Input:**
- `tagger`: Trained HMM tagger
- `test_data`: List of tagged test sentences

**Output:** Dictionary with per-tag statistics `{tag: {'correct': count, 'total': count}}`

In [2]:
def evaluate_tagger(tagger, test_data):
    """
    Evaluate tagger and return per-tag accuracy.

    Args:
        tagger: Trained HMM tagger
        test_data: List of tagged sentences for testing

    Returns:
        dict of {tag: {'correct': count, 'total': count}}
    """
    # TODO: Implement this function
    #
    # Steps:
    # 1. Initialize tag_stats as defaultdict(lambda: {'correct': 0, 'total': 0})
    # 2. For each sentence in test_data:
    #    a. Extract words and true tags
    #    b. Use tagger.tag(words) to get predicted tags
    #    c. Compare true vs predicted tags
    #    d. Update tag_stats accordingly
    # 3. Handle exceptions (if tagging fails, count all as incorrect)
    # 4. Return tag_stats


def evaluate_tagger(tagger, test_data):
    """
    Evaluate tagger and return per-tag accuracy.
    """
    # 1. Initialize stats container
    # When we access a new tag, it automatically starts with 0 correct and 0 total
    tag_stats = defaultdict(lambda: {'correct': 0, 'total': 0})

    # 2. Loop through each sentence in the test data
    for sentence in test_data:
        # Separate the words from the tags
        # sentence input: [('The', 'DET'), ('cat', 'NOUN')]
        # words: ['The', 'cat']
        # true_tags: ['DET', 'NOUN']
        words = [word for word, tag in sentence]
        true_tags = [tag for word, tag in sentence]

        try:
            # b. Use tagger.tag(words) to get predictions
            # The tagger returns a list of tuples: [('The', 'DET'), ('cat', 'VERB')]
            predicted_pairs = tagger.tag(words)

            # Extract just the tags from the prediction
            predicted_tags = [tag for word, tag in predicted_pairs]

            # c. Compare true vs predicted tags
            # zip() allows us to loop through both lists at the same time
            for true_tag, pred_tag in zip(true_tags, predicted_tags):#creates a tuple list

                # d. Update tag_stats
                # Always increment the total count for the TRUE tag
                tag_stats[true_tag]['total'] += 1

                # Only increment 'correct' if the prediction matches the truth
                if true_tag == pred_tag:
                    tag_stats[true_tag]['correct'] += 1

        except Exception as e:
            # 3. Handle exceptions
            # If the tagger crashes on a sentence (e.g., unknown symbol),
            # we count every tag in that sentence as a "miss" (incorrect).
            for tag in true_tags:
                tag_stats[tag]['total'] += 1
                # We do NOT increment 'correct' here

    # 4. Return the result
    return dict(tag_stats)


## 1.4: Semi-supervised HMM Training (20 points)

Implement semi-supervised HMM training that combines a small amount of labelled data (1%) with larger unlabelled data (99%).

**Steps:**
1. Split data: 1% tagged, 99% untagged
2. Train supervised model on 1% tagged data
3. Use this model to initialise Baum-Welch on 99% untagged data
4. Refine with unsupervised learning

In [4]:
def train_semi_supervised(tagged_data, percent_tagged=1.0):
    """
    Train HMM with a mix of tagged and untagged data.

    Args:
        tagged_data: List of tagged sentences
        percent_tagged: Percentage of data to keep tags (default 1.0%)

    Returns:
        Trained HMM tagger
    """
    # TODO: Implement semi-supervised training
    #
    # Steps:
    # 1. Calculate split index: int(len(tagged_data) * (percent_tagged / 100.0))
    #    Ensure at least 1 sentence is tagged
    #
    # 2. Split data:
    #    tagged_portion = tagged_data[:split_idx]
    #    untagged_portion = remove_tags(tagged_data[split_idx:])
    #
    # 3. Extract states and symbols from ALL data
    #    NOTE: In fact, we should have used only the tagged_data for this,
    #    but we are not sure if that portion includes all the tags and symbols
    #
    # 4. Initialize trainer with states and symbols using
    #    hmm.HiddenMarkovModelTrainer
    #
    # 5. Train supervised model on small tagged data using trainer.train_supervised
    #
    # 6. Refine with Baum-Welch on untagged data using trainer.train_unsupervised
    #
    # 7. Return refined_tagger (or error if no untagged data or no tagged data)

# 1. Calculate split index
    total_len = len(tagged_data)
    split_idx = int(total_len * (percent_tagged / 100.0))

    # Safety check: We need at least one labeled sentence to start the process
    if split_idx < 1 and total_len > 0:
        split_idx = 1

    print(f"Splitting: {split_idx} tagged, {total_len - split_idx} untagged.")

    # 2. Split data
    tagged_portion = tagged_data[:split_idx]

    # IMPORTANT: We must use the remove_tags function you wrote earlier
    # to simulate unlabelled data for the rest of the corpus
    untagged_portion = remove_tags(tagged_data[split_idx:])

    # 3. Extract states and symbols from ALL data
    # We scan the entire dataset (even the parts we will "untag" later)
    # to ensure the HMM knows every possible word (Symbol) and tag (State) exists.
    # If we don't do this, the unsupervised trainer will crash on 'unknown' words.
    all_states = set()
    all_symbols = set()

    for sent in tagged_data:
        for word, tag in sent:
            all_states.add(tag)
            all_symbols.add(word)

    # 4. Initialize trainer
    # We must explicitly provide the full list of states and symbols
    trainer = hmm.HiddenMarkovModelTrainer(
        states=list(all_states),
        symbols=list(all_symbols)
    )

    # 5. Train supervised model on the small tagged portion
    # This creates our "seed" model with rough probabilities
    print("Step 1: Training initial supervised model...")
    supervised_model = trainer.train_supervised(tagged_portion)

    # 6. Refine with Baum-Welch on untagged data
    # If we have untagged data, we use the supervised_model to initialize
    # the unsupervised training. This is the "Semi-Supervised" magic.
    if len(untagged_portion) > 0:
        print("Step 2: Refining with unsupervised learning (this may take time)...")

        # We pass model=supervised_model. This tells NLTK:
        # "Don't start with random guesses. Start with what you learned in Step 1."
        refined_model = trainer.train_unsupervised(
            untagged_portion,
            model=supervised_model,
            max_iterations= 5 # Optional: limit iterations to save time during debug
        )
        return refined_model
    else:
        # If percent_tagged was 100%, just return the supervised model
        return supervised_model

## 1.5: 5-Fold Cross-Validation (25 Points)

Implement 5-fold cross-validation to train and evaluate all three models. This ensures robust performance estimates.

**Steps:**
1. Split data into 5 folds
2. For each fold:
   - Train
    - supervised (`trainer.train_supervised()`)
    - unsupervised (`trainer.train_unsupervised()`)
    - semi-supervised models (`train_semi_supervised()`)
   - Evaluate on test fold
   - Accumulate results
3. Calculate average accuracy across all folds

In [10]:
# Prepare for 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=seed)

# Storage for results across folds
results = {
    'unsupervised': defaultdict(lambda: {'correct': 0, 'total': 0}),
    'supervised': defaultdict(lambda: {'correct': 0, 'total': 0}),
    'semi_supervised': defaultdict(lambda: {'correct': 0, 'total': 0})
}
fold_num = 0

# TODO: Complete the cross-validation loop

for train_idx, test_idx in kf.split(all_data):#kf splits the all data we define at the beginning
    fold_num += 1 #&&&&&&&&&
    print(f"{'='*60}")
    print(f"FOLD {fold_num}")
    print(f"{'='*60}")

    # Split data to train_data and test_data
    # ...
    train_data = [all_data[i] for i in train_idx]  #&&&&&&
    test_data = [all_data[i] for i in test_idx]


    print(f"Training set: {len(train_data)} sentences")
    print(f"Test set: {len(test_data)} sentences")

    # Extract all_tags and all_symbols from training data
    #  ...
    current_states = set()   #&&&&&&&&&&&
    current_symbols = set()
    for sent in train_data:
        for word, tag in sent:
            current_states.add(tag)
            current_symbols.add(word) # Keep casing as is from data

    # Convert to lists for the trainer
    list_states = list(current_states)
    list_symbols = list(current_symbols)  #&&&&&&&&&&&

    # 1. Unsupervised HMM (Baum-Welch)
    print("\n1. Training Unsupervised HMM (Baum-Welch)...")
    try:
        # 1.1 Convert to untagged format

        # 1.2 Initialize trainer with states and symbols

        # 1.3 Train unsupervised

        # 1.4 Evaluate unsupervised tagger and update the results dict

        # 1.1 Convert to untagged format
        untagged_train = remove_tags(train_data)

        # 1.2 Initialize trainer
        trainer_unsup = hmm.HiddenMarkovModelTrainer(states=list_states, symbols=list_symbols)

        # 1.3 Train unsupervised (Limit iterations to save time)
        # BU SATIR EKSİKTİ, EKLENDİ:
        tagger_unsup = trainer_unsup.train_unsupervised(untagged_train, max_iterations=5)

        # 1.4 Evaluate
        fold_stats = evaluate_tagger(tagger_unsup, test_data)

        # Accumulate results
        for tag, counts in fold_stats.items():
            results['unsupervised'][tag]['correct'] += counts['correct']
            results['unsupervised'][tag]['total'] += counts['total']

        print("Unsupervised training complete")
    except Exception as e:
        print(f"ERROR: Unsupervised training failed: {e}")
        import traceback
        traceback.print_exc()

    # 2. Supervised HMM
    print("\n2. Training Supervised HMM...")
    try:
        # 1.1 Initialize trainer with states and symbols
        trainer_sup = hmm.HiddenMarkovModelTrainer(states=list_states, symbols=list_symbols)

        # 1.2 Train supervised
        tagger_sup = trainer_sup.train_supervised(train_data)

        # 1.3 Evaluate supervised tagger and update the results dict
        fold_stats = evaluate_tagger(tagger_sup, test_data)

        # Accumulate results
        for tag, counts in fold_stats.items():
            results['supervised'][tag]['correct'] += counts['correct']
            results['supervised'][tag]['total'] += counts['total']
        # 1.1 Initialize trainer with states and symbols

        # 1.2 Train supervised

        # 1.3 Evaluate supervised tagger and update the results dict

        print("Supervised training complete")
    except Exception as e:
        print(f"ERROR: Supervised training failed: {e}")
        import traceback
        traceback.print_exc()

    # 3. Semi-supervised HMM (1% tagged, 99% untagged)
    print("\n3. Training Semi-Supervised HMM (1% tagged, 99% untagged)...")
    try:
        # 1.1 Train semi-supervised

        # 1.3 Evaluate semi-supervised tagger and update the results dict


        # 1.1 Train semi-supervised
        # We pass the custom function we wrote earlier
        # Note: We don't need to manually init the trainer here, the function does it.
        tagger_semi_sup = train_semi_supervised(train_data, percent_tagged=1.0)

        # 1.3 Evaluate semi-supervised tagger and update the results dict
        fold_stats = evaluate_tagger(tagger_semi_sup, test_data)

        # Accumulate results
        for tag, counts in fold_stats.items():
            results['semi_supervised'][tag]['correct'] += counts['correct']
            results['semi_supervised'][tag]['total'] += counts['total']

        print("Semi-supervised training complete")
    except Exception as e:
        print(f"ERROR: Semi-supervised training failed: {e}")
        import traceback
        traceback.print_exc()


FOLD 1
Training set: 4000 sentences
Test set: 1000 sentences

1. Training Unsupervised HMM (Baum-Welch)...
iteration 0 logprob -1190490.3323746119
iteration 1 logprob -878083.5835181965
iteration 2 logprob -876624.6737700262
iteration 3 logprob -875310.256632989
iteration 4 logprob -873853.671049464
Unsupervised training complete

2. Training Supervised HMM...


/usr/local/lib/python3.12/dist-packages/nltk/tag/hmm.py:335: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
/usr/local/lib/python3.12/dist-packages/nltk/tag/hmm.py:333: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
/usr/local/lib/python3.12/dist-packages/nltk/tag/hmm.py:363: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])


Supervised training complete

3. Training Semi-Supervised HMM (1% tagged, 99% untagged)...
Splitting: 40 tagged, 3960 untagged.
Step 1: Training initial supervised model...
Step 2: Refining with unsupervised learning (this may take time)...
iteration 0 logprob -4.484600000000029e+304
iteration 1 logprob -6.391436595342868e+289
iteration 2 logprob -892037.196691362
iteration 3 logprob -865828.7819085391
iteration 4 logprob -863749.4419701553


/usr/local/lib/python3.12/dist-packages/nltk/tag/hmm.py:331: RuntimeWarning: overflow encountered in cast
  P[i] = self._priors.logprob(si)


Semi-supervised training complete
FOLD 2
Training set: 4000 sentences
Test set: 1000 sentences

1. Training Unsupervised HMM (Baum-Welch)...
iteration 0 logprob -1193534.6787396027
iteration 1 logprob -876247.977715901
iteration 2 logprob -874801.011138248
iteration 3 logprob -873255.0167542782
iteration 4 logprob -871345.2619476303
Unsupervised training complete

2. Training Supervised HMM...
Supervised training complete

3. Training Semi-Supervised HMM (1% tagged, 99% untagged)...
Splitting: 40 tagged, 3960 untagged.
Step 1: Training initial supervised model...
Step 2: Refining with unsupervised learning (this may take time)...
iteration 0 logprob -4.47010000000003e+304
iteration 1 logprob -6.2580809189905805e+289
iteration 2 logprob -892858.1622283141
iteration 3 logprob -863630.3271532262
iteration 4 logprob -861515.4034895102
Semi-supervised training complete
FOLD 3
Training set: 4000 sentences
Test set: 1000 sentences

1. Training Unsupervised HMM (Baum-Welch)...
iteration 0 logp

**Question 1.1:** In an unsupervised HMM with 12 hidden states, you replace the intended PoS tags with meaningless labels like S1–S12. How would this change affect the model’s learned behaviour and its evaluation accuracy? (4 points, 3-4 sentences)

**[YOUR ANSWER HERE]**

**Question 1.2:** You initialize a semi-supervised HMM with a fully supervised model and then run Baum–Welch only on unlabeled data. How can this additional unsupervised training change the model’s behaviour and accuracy? (4 points, 3-4 sentences)

**[YOUR ANSWER HERE]**

**Question 1.3:** Consider the following four changes to an unsupervised or semi-supervised HMM for PoS tagging. For each one, state whether it would make training faster, slower, or have no significant effect, and justify your choice in one sentence. (4 points, 4 sentences)
- Reducing the number of hidden states from 12 to 6.
- Initializing the model with a fully supervised HMM instead of random parameters.
- Increasing the size of the unlabeled corpus by a factor of 5.
- Replacing full EM with a fixed small number of EM iterations (e.g., exactly 3 passes).

**[YOUR ANSWER HERE]**

## 1.6: Display Results (5 points)

Display the average across folds results in a formatted table showing
- tag frequency (%)
- per-tag accuracy (%)
- overall accuracy (%)


1.   List item
2.   List item


for all three models.

In [11]:
# TODO: Calculate and display results
#


# Iterate through each model type (Supervised, Unsupervised, Semi-supervised)
# We sort the keys to ensure a consistent order, or manually specify the list
model_order = ['supervised', 'unsupervised', 'semi_supervised']

print(f"{'='*80}")
print(f"{'FINAL EVALUATION RESULTS (Average across 5 Folds)':^80}")
print(f"{'='*80}\n")

for model_name in model_order:
    # Check if we have data for this model
    if model_name not in results:
        continue

    data = results[model_name]

    # 1. Calculate Overall Statistics
    total_correct = sum(tag_data['correct'] for tag_data in data.values())
    total_tokens = sum(tag_data['total'] for tag_data in data.values())

    # Avoid division by zero
    overall_accuracy = (total_correct / total_tokens * 100) if total_tokens > 0 else 0.0

    # 2. Print Header for the Model
    print(f"MODEL: {model_name.upper().replace('_', ' ')}")
    print(f"{'-'*45}")
    print(f"{'TAG':<12} | {'FREQ (%)':<12} | {'ACCURACY (%)':<12}")
    print(f"{'-'*45}")

    # 3. Calculate and Print Per-Tag Statistics
    # Sort tags alphabetically for cleaner reading
    sorted_tags = sorted(data.keys())

    for tag in sorted_tags:
        stats = data[tag]
        tag_total = stats['total']
        tag_correct = stats['correct']

        if tag_total > 0:
            # Frequency: How often does this tag appear in the whole text?
            frequency = (tag_total / total_tokens) * 100

            # Accuracy: How often did we get THIS specific tag right?
            accuracy = (tag_correct / tag_total) * 100

            print(f"{tag:<12} | {frequency:>10.2f}% | {accuracy:>10.2f}%")
        else:
            print(f"{tag:<12} | {'0.00%':>10} | {'N/A':>10}")

    # 4. Print Overall Accuracy Footer
    print(f"{'-'*45}")
    print(f"{'OVERALL':<12} | {'100.00%':>10} | {overall_accuracy:>10.2f}%")
    print("\n" + "="*80 + "\n")

               FINAL EVALUATION RESULTS (Average across 5 Folds)                

MODEL: SUPERVISED
---------------------------------------------
TAG          | FREQ (%)     | ACCURACY (%)
---------------------------------------------
.            |      11.68% |      33.45%
ADJ          |       6.82% |      97.67%
ADP          |      12.32% |      40.17%
ADV          |       3.42% |      43.48%
CONJ         |       2.73% |      35.10%
DET          |      11.42% |      49.04%
NOUN         |      30.17% |      36.18%
NUM          |       2.06% |      39.58%
PRON         |       2.56% |      56.18%
PRT          |       2.25% |      40.83%
VERB         |      14.47% |      44.73%
X            |       0.09% |      23.23%
---------------------------------------------
OVERALL      |    100.00% |      44.15%


MODEL: UNSUPERVISED
---------------------------------------------
TAG          | FREQ (%)     | ACCURACY (%)
---------------------------------------------
.            |      11.68% |  

## 1.7: Sample Predictions

Test the models on sample sentences to see how it performs.

In [12]:
# Print sample predictions from the last fold
print("Sample predictions from last fold:")
sample_sentences = [
    "Today is a good day .",
    "Joe met Joanne in Delhi .",
    "Time flies like an arrow .",
    "The good , the bad , the ugly went to a bar ."
]

print(f"{'='*10} Spervised HMM Tagger {'='*10}")
for sent in sample_sentences:
    try:
        tagged = tagger_sup.tag(sent.split())
        print(f"  {sent}")
        print(f"  → {tagged}\n")
    except Exception as e:
        print(f"  {sent}")
        print(f"ERROR: Tagging failed: {e}\n")

print(f"{'='*10} Unspervised HMM Tagger {'='*10}")
for sent in sample_sentences:
    try:
        tagged = tagger_unsup.tag(sent.split())
        print(f"  {sent}")
        print(f"  → {tagged}\n")
    except Exception as e:
        print(f"  {sent}")
        print(f"ERROR: Tagging failed: {e}\n")

print(f"{'='*10} Semi-spervised HMM Tagger {'='*10}")
for sent in sample_sentences:
    try:
        tagged = tagger_semi_sup.tag(sent.split())
        print(f"  {sent}")
        print(f"  → {tagged}\n")
    except Exception as e:
        print(f"  {sent}")
        print(f"ERROR: Tagging failed: {e}\n")

Sample predictions from last fold:
========== Spervised HMM Tagger ==========
  Today is a good day .
  → [('Today', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('good', 'ADJ'), ('day', 'NOUN'), ('.', '.')]

  Joe met Joanne in Delhi .
  → [('Joe', 'NOUN'), ('met', 'VERB'), ('Joanne', 'NOUN'), ('in', 'ADP'), ('Delhi', 'NOUN'), ('.', '.')]

  Time flies like an arrow .
  → [('Time', 'NOUN'), ('flies', 'VERB'), ('like', 'ADP'), ('an', 'DET'), ('arrow', 'ADJ'), ('.', 'ADJ')]

  The good , the bad , the ugly went to a bar .
  → [('The', 'DET'), ('good', 'ADJ'), (',', '.'), ('the', 'DET'), ('bad', 'ADJ'), (',', '.'), ('the', 'DET'), ('ugly', 'ADJ'), ('went', 'VERB'), ('to', 'ADP'), ('a', 'DET'), ('bar', 'NOUN'), ('.', '.')]

========== Unspervised HMM Tagger ==========
  Today is a good day .
  → [('Today', 'CONJ'), ('is', 'ADV'), ('a', 'ADP'), ('good', 'VERB'), ('day', 'PRON'), ('.', 'ADP')]

  Joe met Joanne in Delhi .
  → [('Joe', 'ADJ'), ('met', 'DET'), ('Joanne', 'CONJ'), ('in', 'ADV'), ('D

---

# Task 2: Text Generation with HMMs (24 points)

In this part, you will use a trained HMM to generate text. The idea is to sample from the learned transition and emission probabilities to create new sequences.

## 2.1: Train HMM Model for Generation

First, train a supervised HMM model on the Brown corpus for text generation. We'll normalise words to lowercase for better generation.

In [13]:
def train_hmm_model(num_sentences=5000):
    """Train an HMM model on Brown corpus for text generation"""
    print("Loading Brown corpus...")
    tagged_sents = list(brown.tagged_sents(tagset="universal"))[:num_sentences]

    print(f"Training HMM on {len(tagged_sents)} sentences...")

    # Extract states and symbols
    all_tags = set()
    all_symbols = set()
    for sent in tagged_sents:
        for word, tag in sent:
            all_tags.add(tag)
            all_symbols.add(word.lower())  # Normalise to lowercase

    # Normalise training data to lowercase
    normalized_sents = []
    for sent in tagged_sents:
        normalized_sents.append([(word.lower(), tag) for word, tag in sent])

    # Train the model
    trainer = hmm.HiddenMarkovModelTrainer(
        states=list(all_tags),
        symbols=list(all_symbols)
    )

    tagger = trainer.train_supervised(normalized_sents)

    print("Training complete!")
    return tagger

# Train the model
tagger_gen = train_hmm_model(num_sentences=20000)

# Show model statistics
print(f"\nModel Statistics:")
print(f"  - Number of states (POS tags): {len(tagger_gen._states)}")
print(f"  - Number of symbols (words): {len(tagger_gen._symbols)}")
print(f"  - States: {', '.join(sorted(tagger_gen._states))}")

Loading Brown corpus...
Training HMM on 20000 sentences...
Training complete!

Model Statistics:
  - Number of states (POS tags): 12
  - Number of symbols (words): 30743
  - States: ., ADJ, ADP, ADV, CONJ, DET, NOUN, NUM, PRON, PRT, VERB, X


## 2.2: Implement State Sampling (7 points)

Implement the `sample_state()` function that samples the next POS tag based on transition probabilities.

**Temperature parameter:** Controls randomness
- Lower temperature (e.g., 0.5): More conservative, follows high-probability transitions
- Higher temperature (e.g., 2.0): More creative, explores diverse transitions

In [14]:
def sample_state(tagger, current_state, temperature=1.0):
    """
    Sample next state from transition distribution.

    Args:
        tagger: Trained HMM tagger
        current_state: Current POS tag
        temperature: Controls randomness (default 1.0)

    Returns:
        Next POS tag (state)
    """
    # TODO: Implement state sampling
    #
    # Steps:
    # 1. Get all possible states from tagger._states
    #
    # 2. For each state, get transition probability from current_state
    #    using: tagger._transitions[current_state].logprob(state)
    #
    # 3. Apply temperature scaling: probs = probs / temperature
    #
    # 4. Convert from log2 probabilities to regular probabilities: 2 ** probs
    #
    # 5. Normalise probabilities to sum to 1
    #
    # 6. Sample using np.random.choice(states, p=probs)

    # 1. Get all possible states from tagger._states
    # We convert to a list to ensure consistent ordering for numpy
    states = list(tagger._states)

    # 2. Get transition probabilities (Log Base 2)
    # We use a list comprehension to get the probability of moving from
    # current_state -> s for every possible state 's'
    log_probs = np.array([tagger._transitions[current_state].logprob(s) for s in states])

    # 3. Apply temperature scaling
    # We divide the log probs by temperature.
    # Avoid division by zero by setting a lower bound if needed,
    # though usually we assume temp > 0.
    if temperature <= 0: temperature = 1e-10
    scaled_log_probs = log_probs / temperature

    # 4. Convert from log2 probabilities to regular probabilities
    # NLTK uses log base 2, so we calculate 2 to the power of x
    probs = 2.0 ** scaled_log_probs

    # 5. Normalise probabilities to sum to 1
    total_prob = np.sum(probs)

    # Handle edge case: if current_state has no valid transitions (sum is 0),
    # fall back to uniform distribution to prevent crash
    if total_prob == 0:
        probs = np.ones(len(states)) / len(states)
    else:
        probs = probs / total_prob

    # 6. Sample using np.random.choice
    next_state = np.random.choice(states, p=probs)

    return next_state

## 2.3: Implement Word Sampling (7 points)

Implement the `sample_word()` function that samples a word given a POS tag based on emission probabilities.

In [15]:
def sample_word(tagger, state, temperature=1.0):
    """
    Sample word from output distribution of given state.

    Args:
        tagger: Trained HMM tagger
        state: Current POS tag
        temperature: Controls randomness (default 1.0)

    Returns:
        Sampled word
    """
    # TODO: Implement word sampling
    #
    # Steps:
    # 1. Get all possible symbols (words) from tagger._symbols
    #
    # 2. For each symbol, get emission probability from state
    #    using: tagger._outputs[state].logprob(symbol)
    #
    # 3. Apply temperature scaling: probs = probs / temperature
    #
    # 4. Convert from log2 probabilities to regular probabilities: 2 ** probs
    #
    # 5. Handle infinities (words with zero probability) using np.nan_to_num
    #
    # 6. Normalise probabilities to sum to 1
    #    (if all are zero, use uniform distribution)
    #
    # 7. Sample using np.random.choice(symbols, p=probs)

# 1. Get all possible symbols (words) from tagger._symbols
    # Convert to list to ensure index alignment with probabilities
    symbols = list(tagger._symbols)

    # 2. Get emission probabilities (Log Base 2)
    # tagger._outputs[state] gives the probability distribution for that state.
    # We calculate the log probability for every word in the vocabulary.
    log_probs = np.array([tagger._outputs[state].logprob(word) for word in symbols])

    # 3. Apply temperature scaling
    if temperature <= 0: temperature = 1e-10
    scaled_log_probs = log_probs / temperature

    # 4. Convert from log2 probabilities to regular probabilities
    # 2^x converts NLTK's log2 notation back to standard probability space
    probs = 2.0 ** scaled_log_probs

    # 5. Handle infinities and NaNs
    # Words with 0 probability have log_prob of -inf.
    # 2**-inf is 0, but sometimes numpy generates nan or very small garbage values.
    # This cleans the array.
    probs = np.nan_to_num(probs)

    # 6. Normalise probabilities to sum to 1
    total_prob = np.sum(probs)

    if total_prob == 0:
        # If the state has no emissions (unlikely but possible in broken models),
        # return a random word to avoid crashing
        probs = np.ones(len(symbols)) / len(symbols)
    else:
        probs = probs / total_prob

    # 7. Sample using np.random.choice
    word = np.random.choice(symbols, p=probs)

    return word

## 2.4: Implement Text Generation (10 points)

Implement the `generate_text_from_word()` function that generates text starting from a given word.

In [16]:
def generate_text_from_word(tagger, start_word, length=20, temperature=1.0):
    """
    Generate text starting with a given word using the HMM model.

    Args:
        tagger: Trained HMM tagger
        start_word: Word to start the generation
        length: Number of words to generate
        temperature: Controls randomness (higher = more random)

    Returns:
        List of generated words
    """
    # TODO: Implement text generation
    #
    # Steps:
    # 1. Normalise start_word to lowercase
    #
    # 2. Check if start_word is in vocabulary (tagger._symbols)
    #    - If not, find similar words or use random word
    #
    # 3. Initialise generated list with start_word
    #
    # 4. Infer the most likely tag for start_word:
    #    - For each state, get emission probability
    #    - Choose state with highest probability
    #
    # 5. Generate subsequent words in a loop:
    #    - Sample next state using sample_state()
    #    - Sample word from that state using sample_word()
    #    - Append word to generated list
    #    - Update current_state
    #
    # 6. Return generated list

# 1. Normalise start_word to lowercase
    # This matches the training data format
    current_word = start_word.lower()

    # 2. Check if start_word is in vocabulary
    # We convert to a set for faster lookup, though list check works too
    vocab = set(tagger._symbols)

    if current_word not in vocab:
        print(f"Warning: '{current_word}' not in vocabulary. Picking a random start word.")
        # Fallback: Pick a random word from the known vocabulary
        current_word = random.choice(list(vocab))

    # 3. Initialise generated list
    generated_words = [current_word]

    # 4. Infer the most likely tag (state) for the start_word
    # We need to know "What Part of Speech is this word?" to start the chain.
    # We check P(Word | Tag) for every tag and pick the highest one.
    best_state = None
    max_log_prob = -float('inf')

    for state in tagger._states:
        # Get emission probability: P(word | state)
        # Note: logprob returns -inf if probability is 0
        prob = tagger._outputs[state].logprob(current_word)

        if prob > max_log_prob:
            max_log_prob = prob
            best_state = state

    # Safety fallback if something goes wrong (e.g. word has 0 prob in all states)
    if best_state is None:
        best_state = random.choice(list(tagger._states))

    current_state = best_state

    # 5. Generate subsequent words in a loop
    # We start range at 1 because we already have the 0th word
    for _ in range(length - 1):

        # A. Sample next state (Transition: State -> State)
        next_state = sample_state(tagger, current_state, temperature)

        # B. Sample word from that state (Emission: State -> Word)
        next_word = sample_word(tagger, next_state, temperature)

        # C. Append and Update
        generated_words.append(next_word)
        current_state = next_state

    # 6. Return generated list
    return generated_words

## 2.6: Test Text Generation

Now test your text generation functions with different starting words and temperatures.

In [17]:
# Test with example words
print("="*70)
print("EXAMPLE GENERATIONS")
print("="*70)

example_words = ["the", "dog", "running", "beautiful", "computer", "yesterday"]

for word in example_words:
    print(f"\n--- Starting with '{word}' ---")
    try:
        words = generate_text_from_word(tagger_gen, word, length=15, temperature=1.0)
        print(" ".join(words))
    except Exception as e:
        print(f"Error: {e}")

EXAMPLE GENERATIONS

--- Starting with 'the' ---
the one , but patriotic office when such pier by investigation postulated from 9 by

--- Starting with 'dog' ---
dog receiving his less , as the grandiose important borden by the delivering ; a

--- Starting with 'running' ---
running readily good president . he of deep venture vocational accumulation `` if the sedition

--- Starting with 'beautiful' ---
beautiful view , , however am being the larger least russia . alarm as generation

--- Starting with 'computer' ---
computer , a wards , is if hotel ( eclectic file to quarrel the berkman

--- Starting with 'yesterday' ---
yesterday . bed on the perilous to time . an industrial father scuttled inside in


In [22]:
# Test different temperatures
print("\n" + "="*70)
print("TEMPERATURE COMPARISON")
print("="*70)

test_word = "the"
temperatures = [("Conservative", 0.5), ("Balanced", 1.0), ("Creative", 2.0)]

for temp_name, temp_value in temperatures:
    print(f"\n{temp_name} (temp={temp_value}):")
    words = generate_text_from_word(tagger_gen, test_word, length=20, temperature=temp_value)
    print(" ".join(words))


TEMPERATURE COMPARISON

Conservative (temp=0.5):
the year be the way in the most berlin . very is is the president days for the home to

Balanced (temp=1.0):
the plane act , states tests march as political harold signs lines that highwayman sustain but a nov. and grave

Creative (temp=2.0):
the well water borax , '' yet visual sold-out lebanese out rebel amply a ferdinando in any , of ration


**Question 2.1:** How does the temperature parameter affect the quality and creativity of generated text? Provide two specific examples from your outputs. (4 points, 5-6 sentences)

**[YOUR ANSWER HERE]**

# Convert Your Colab Notebook to PDF

### Step 1: Download Your Notebook
- Go to **File → Download → Download .ipynb**
- Save the file to your computer

### Step 2: Upload to Colab
- Click the **📁 folder icon** on the left sidebar
- Click the **upload button**
- Select your downloaded .ipynb file
- Wait for the upload to complete

### Step 3: Run the Code Below
- **Uncomment the cell below** and run the cell
- This will take about 1-2 minutes to install required packages

### Step 4: Enter Notebook Name
- When prompted, type your notebook name (e.g.`gs_000000_as2.ipynb`)
- Press Enter

### The PDF will automatically download to your computer

In [21]:
# Install required packages (this takes about 30 seconds)
print("Installing PDF converter... please wait...")
!apt-get update -qq
!apt-get install -y texlive-xetex texlive-fonts-recommended texlive-plain-generic pandoc > /dev/null 2>&1
!pip install -q nbconvert

print("\n" + "="*50)
print("COLAB NOTEBOOK TO PDF CONVERTER")
print("="*50)
print("\nSTEP 1: Download your notebook")
print("- Go to File → Download → Download .ipynb")
print("- Save it to your computer")
print("\nSTEP 2: Upload it here")
print("- Click the folder icon on the left (📁)")
print("- Click the upload button and select your .ipynb file")
print("- Wait for upload to complete")
print("\nSTEP 3: Enter the filename below")
print("="*50)

# Get notebook name from user
notebook_name = input("\nEnter your notebook name: ")

# Add .ipynb if missing
if not notebook_name.endswith('.ipynb'):
    notebook_name += '.ipynb'

import os
notebook_path = f'/content/{notebook_name}'

# Check if file exists
if not os.path.exists(notebook_path):
    print(f"\n⚠ Error: '{notebook_name}' not found in /content/")
    print("\nMake sure you uploaded the file using the folder icon (📁) on the left!")
else:
    print(f"\n✓ Found {notebook_name}")
    print("Converting to PDF... this may take 1-2 minutes...\n")

    # Convert the notebook to PDF
    !jupyter nbconvert --to pdf "{notebook_path}"

    # Download the PDF
    from google.colab import files
    pdf_name = notebook_name.replace('.ipynb', '.pdf')
    pdf_path = f'/content/{pdf_name}'

    if os.path.exists(pdf_path):
        print("✓ SUCCESS! Downloading your PDF now...")
        files.download(pdf_path)
        print("\n✓ Done! Check your downloads folder.")
    else:
        print("⚠ Error: Could not create PDF")

Installing PDF converter... please wait...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)

COLAB NOTEBOOK TO PDF CONVERTER

STEP 1: Download your notebook
- Go to File → Download → Download .ipynb
- Save it to your computer

STEP 2: Upload it here
- Click the folder icon on the left (📁)
- Click the upload button and select your .ipynb file
- Wait for upload to complete

STEP 3: Enter the filename below

Enter your notebook name: cng463_assignment3

✓ Found cng463_assignment3.ipynb
Converting to PDF... this may take 1-2 minutes...

[NbConvertApp] Converting notebook /content/cng463_assignment3.ipynb to pdf
[NbConvertApp] Writing 118154 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | b

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓ Done! Check your downloads folder.
